### Library

In [58]:
from datetime import datetime, timedelta
import oandapy
import sys
sys.path.append('/home/toshio/projects/fx')
from config import token, acc_id
from lib.inference import Inference
import time

In [59]:
def get_current_state():
    try:
        response = oanda.get_position(acc_id, instrument ='USD_JPY')
        side = response.get('side')
        units = response.get('units')
    except oandapy.OandaError:
        side = 'zero_position'
        units = 0
    return side, units

In [60]:
def order(side, units):
    trade_expire = datetime.utcnow() + timedelta(minutes=10)
    trade_expire = trade_expire.isoformat("T") + "Z"
    
    if side == 'buy':
        price = oanda.get_prices(instruments ='USD_JPY')['prices'][0]['ask']
    elif side == 'sell':
        price = oanda.get_prices(instruments ='USD_JPY')['prices'][0]['bid']
    
    response = oanda.create_order(acc_id,
        instrument='USD_JPY',
        units=units,
        side=side,
        type='limit',
        price=price,
        expiry=trade_expire
    )    

In [64]:
def transaction():
    #現在のポジションを取得
    current_side, current_units = get_current_state()

    #ゼロポジションにする
    if current_side == 'buy':
        print('now you are long position')
        order('sell', current_units)
        print('reset position')
    elif current_side == 'sell':
        print('now you are short position')
        order('buy', current_units)
        print('reset position')
    elif current_side == 'zero_position':
        print('now you are zero position')

    #1時間後の差額を推定する
    inf = Inference()
    pred_delta=float(inf.pred[0][0])
    print('1時間の推定差額は{}'.format(pred_delta))

    if pred_delta >= 0.1:
        print('Buy!')
        order('buy', 100)
    elif pred_delta <= -0.1:
        print('Sell!')
        order('sell', 100)
    elif -0.1 < pred_delta < 0.1:
        print('do nothing')

In [ ]:
oanda = oandapy.API(environment="practice", access_token=token)
#100時間取引する
j = 0
while j < 100:
    j +=1
    now = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S')
    print('------transaction_{0:03} {1}------'.format(j, now))
    transaction()
    time.sleep(3600)

------transaction_001 2018-09-06 11:49:26------
now you are zero position
1時間の推定差額は-0.007457053754478693
do nothing
------transaction_002 2018-09-06 12:49:32------
now you are zero position
1時間の推定差額は-0.006690496113151312
do nothing
------transaction_003 2018-09-06 13:49:36------
now you are zero position
1時間の推定差額は-0.004511792678385973
do nothing
